In [39]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [40]:
pip install pysocks scikit-learn  

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import datetime

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [6]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
#actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
# length_of_array = len(actions)
# print(length_of_array)
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ'])	
# Thirty videos worth of data
no_sequences = 30

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [7]:
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))

            if os.path.exists(file_path):
                res = np.load(file_path)
            else:
                print(f"Error: File not found at {file_path}")
            # # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            # res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
# shape should be 900(30 action 1 action has 30 videos) videos,30 frame each,126(21*3*2) key points in a frame
np.array(sequences).shape

(300, 30, 126)

In [10]:
np.array(labels).shape

(300,)

In [11]:
X = np.array(sequences)

In [12]:
y = to_categorical(labels).astype(int)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [14]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [15]:
# Get the current date in YYYYMMDD format
current_date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Check if the checkpoint directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a latest checkpoint exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Path to save the model weights
    save_weights_only=True,     # Save only the model weights
    verbose=1,                  # Verbosity mode, 1 for progress bar
    save_freq=50          
)



In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
# If a latest checkpoint exists, load the weights and continue training
if latest_checkpoint:
    print(f"Loading weights from checkpoint: {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
# else:
#     if os.path.exists('action.h5'):
#         # Load the pre-trained model from the H5 file
#         model = tf.keras.models.load_model('action.h5')

#         # Get the model summary
#         model.summary()

        # # Get the model weights
        # model_weights = loaded_model.get_weights()
        # model = model.set_weights(model_weights)



In [19]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 1s 879ms/step - loss: 2.3019 - categorical_accuracy: 0.0667
Test Accuracy: 0.07


In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,cp_callback])

Epoch 1/2000
9/9 [==============================] - 4s 28ms/step - loss: 2.2993 - categorical_accuracy: 0.1228
Epoch 2/2000
9/9 [==============================] - 0s 34ms/step - loss: 2.2544 - categorical_accuracy: 0.1544
Epoch 3/2000
9/9 [==============================] - 1s 60ms/step - loss: 2.2615 - categorical_accuracy: 0.2421
Epoch 4/2000
9/9 [==============================] - 1s 71ms/step - loss: 2.2052 - categorical_accuracy: 0.1684
Epoch 5/2000
9/9 [==============================] - 1s 59ms/step - loss: 2.1656 - categorical_accuracy: 0.1544
Epoch 6/2000
3/9 [=========>....................] - ETA: 0s - loss: 1.9890 - categorical_accuracy: 0.1979
Epoch 6: saving model to checkpoints\cp-0006.ckpt
9/9 [==============================] - 0s 51ms/step - loss: 1.8711 - categorical_accuracy: 0.2386
Epoch 7/2000
9/9 [==============================] - 0s 57ms/step - loss: 1.7205 - categorical_accuracy: 0.3439
Epoch 8/2000
9/9 [==============================] - 1s 61ms/step - loss: 1.7632 

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            48896     
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 30)                990       
                                                                 
Total params: 204350 (798.24 KB)
Trainable params: 20435

In [19]:
res = model.predict(X_test)

2/2 [==============================] - 0s 8ms/step


In [20]:
actions[np.argmax(res[4])]

'ට්'

In [21]:
actions[np.argmax(y_test[4])]

'බ්'

In [22]:
model.save('action.keras')